In [125]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

# Import Libraries

In [126]:
import pandas as pd
import re
from gensim.models import Word2Vec
from nltk import word_tokenize
import gensim
from nltk.corpus import stopwords
import nltk
from string import punctuation
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from collections import Counter
from scipy.spatial.distance import cosine
import numpy as np
from gensim.models import Word2Vec, KeyedVectors

In [45]:
import logging
#used for word2vec warnings
# Disable warning messages from being printed to the console
logging.disable(logging.WARNING)

# SKILL

## Reading Skills

In [136]:
df_skill = pd.read_csv("ML_skills.csv", header=None, sep=";")
df_skill["skills"] = [row.split("{")[2] for row in df_skill[0]]
df_skill= df_skill.drop(columns=0,axis=1)
df_skill.head()

,skills
0,"'name': 'Anindya Saha', 'skill': ['Java', 'Pyt..."
1,"'name': 'M. Ozan Tezcan', 'skill': ['Java', 'C..."
2,"'name': 'Clare C.', 'skill': ['Software Engine..."
3,"'name': 'Shaswat Babhulgaonkar', 'skill': ['So..."
4,"'name': 'Sai Gurrapu', 'skill': ['Machine Lear..."


In [137]:
df_skill['skills'] = df_skill['skills'].apply(lambda x: x.split(", 'skill'")[1])
def clean(text):
    cleaned_text = text[2:-3]
    a = cleaned_text.split(", ")
    a = [i[1:-1] for i in a]
    return a

df_skill["skills"] = df_skill["skills"].apply(clean)
f = lambda x: ','.join(map(str, x)) if isinstance(x, list) else x
df_skill['skills'] = df_skill['skills'].apply(f)
df_skill['skills'] = df_skill['skills'].apply(lambda x: x.replace("'", ''))
df_skill['skills'] = df_skill['skills'].str.lower()

In [138]:
def remove_duplicate_words(string):
  words = string.split(",")
  unique_words = []

  # iterate through the list of words
  for word in words:
    # if the word is not in the list of unique words, append it
    if word not in unique_words:
      unique_words.append(word)

  # join the list of unique words into a string and return it
  return ", ".join(unique_words)

In [139]:
df_skill["skills"] = df_skill["skills"].apply(remove_duplicate_words) #same skills in row
df_skill

,skills
0,"java, python, machine learning, data mining, r..."
1,"java, c++, matlab, digital signal processors, ..."
2,"software engineering, artificial intelligence ..."
3,"software development, machine learning, deep l..."
4,"machine learning, leadership, research and dev..."
...,...
92,"end-to-end text-to-speech, end-to-end speech r..."
93,"mapreduce, machine learning engineer ii (corte..."
94,"matlab, machine learning, python, c++, c, prog..."
95,"software engineering, artificial intelligence ..."


## Frequency of Skills

In [140]:
df_skill['words'] = df_skill['skills'].apply(lambda x: x.split(","))
word_freq = {}

In [141]:
for row in df_skill['words']:
    for word in row:
        if word not in word_freq:
            word_freq[word] = 1
        else:
            word_freq[word] += 1
for word, freq in word_freq.items():
    print(f'{word}: {freq}')

java: 9
 python: 63
 machine learning: 56
 data mining: 30
 redis: 2
 natural language processing: 11
 hadoop: 19
 sqoop: 1
 hive: 8
 apache spark: 17
 sql: 51
 drools: 1
 jbpm: 1
 j2ee web services: 1
 ejb: 1
 spring: 1
 oracle: 2
 eclipse: 3
 hibernate: 3
 mysql: 27
 jprofiler: 1
 jmx: 1
 web services: 5
 soa: 1
 ibm rational software architect: 1
 sdlc: 1
 multithreading: 1
 solution architecture: 1
 cloud computing: 6
 ooad: 1
 junit: 2
 maven: 2
 github: 5
 tomcat: 1
 uml: 1
 jboss application server: 1
 weblogic: 1
 pl/sql: 4
 open source: 1
 enterprise software: 1
 c++: 59
 matlab: 40
 digital signal processors: 2
 digital signal processing: 1
 xilinx: 1
 c: 44
 microprocessors: 1
 logisim: 1
 avr: 1
 microsoft office: 27
 deep learning: 41
 computer vision: 23
software engineering: 2
 artificial intelligence (ai): 17
 software development: 18
 front-end development: 2
 algorithms: 31
 object oriented design: 2
 object-oriented programming (oop): 3
 web applications: 4
 data sci

In [142]:
sorted_word_freq = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
top_15_words = sorted_word_freq[:15]
for word, freq in top_15_words:
    print(f'{word}: {freq}')

#java ent edition, java web services,core java,java 8

 python: 63
 java: 60
 c++: 59
 machine learning: 56
 sql: 51
 c: 44
 deep learning: 41
 matlab: 40
 data analysis: 35
 r: 34
 tensorflow: 32
 algorithms: 31
 data mining: 30
 linux: 29
 mysql: 27


In [143]:
df_skill= df_skill.drop(columns="words",axis=1)


## Preprocess

In [144]:
lemmatizer = WordNetLemmatizer()
def tokenize_skills(string):
    tokens = re.split(r'[,\s]+', string)
    tokens = [token for token in tokens if token]# Remove any empty tokens
    return tokens
# Remove stop words
def remove_stop_words_skills(tokens):
    clean_tokens = [token for token in tokens if token.lower() not in stopwords.words('english')]
    return clean_tokens

def lemmatize_word(word):
  return lemmatizer.lemmatize(word)
def lemmatize_list(words):
  lemmas = []
  for word in words:
    lemma = lemmatize_word(word)
    lemmas.append(lemma)
  return lemmas

In [145]:
df_skill.head()

,skills
0,"java, python, machine learning, data mining, r..."
1,"java, c++, matlab, digital signal processors, ..."
2,"software engineering, artificial intelligence ..."
3,"software development, machine learning, deep l..."
4,"machine learning, leadership, research and dev..."


In [146]:
df_skill['skills'] = df_skill['skills'].apply(tokenize_skills)
df_skill['skills'] = df_skill['skills'].apply(lemmatize_list)
df_skill['skills'] = df_skill['skills'].apply(remove_stop_words_skills)

In [147]:
df_skill.head()

,skills
0,"[java, python, machine, learning, data, mining..."
1,"[java, c++, matlab, digital, signal, processor..."
2,"[software, engineering, artificial, intelligen..."
3,"[software, development, machine, learning, dee..."
4,"[machine, learning, leadership, research, deve..."


## Skill model

In [148]:
corpus_skill = df_skill['skills'] .tolist()

In [149]:
len(corpus_skill)

97

In [151]:
corpus_skill

[['java',
  'python',
  'machine',
  'learning',
  'data',
  'mining',
  'redis',
  'natural',
  'language',
  'processing',
  'hadoop',
  'sqoop',
  'hive',
  'apache',
  'spark',
  'sql',
  'drool',
  'jbpm',
  'j2ee',
  'web',
  'service',
  'ejb',
  'spring',
  'oracle',
  'eclipse',
  'hibernate',
  'mysql',
  'jprofiler',
  'jmx',
  'web',
  'service',
  'soa',
  'ibm',
  'rational',
  'software',
  'architect',
  'sdlc',
  'multithreading',
  'solution',
  'architecture',
  'cloud',
  'computing',
  'ooad',
  'junit',
  'maven',
  'github',
  'tomcat',
  'uml',
  'jboss',
  'application',
  'server',
  'weblogic',
  'pl/sql',
  'open',
  'source',
  'enterprise',
  'software'],
 ['java',
  'c++',
  'matlab',
  'digital',
  'signal',
  'processor',
  'digital',
  'signal',
  'processing',
  'xilinx',
  'c',
  'microprocessor',
  'logisim',
  'avr',
  'microsoft',
  'office',
  'machine',
  'learning',
  'deep',
  'learning',
  'computer',
  'vision'],
 ['software',
  'engineering

In [152]:
model_skill = Word2Vec(corpus_skill, size=10, window=1, min_count=1, workers=4)


In [153]:
model_skill.train(corpus_skill, total_examples=len(corpus_skill), epochs=5)

(11568, 18530)

In [154]:
similar_words = model_skill.wv.most_similar('machine')
similar_words

[('teamwork', 0.9870043396949768),
 ('learning', 0.9817779064178467),
 ('c', 0.97763991355896),
 ('analysis', 0.9736402034759521),
 ('management', 0.9707362651824951),
 ('java', 0.9704086184501648),
 ('r', 0.9683184623718262),
 ('c++', 0.9674683809280396),
 ('testing', 0.9672423601150513),
 ('computer', 0.9657800197601318)]

# COURSE OUTCOMES

## Reading Outcomes

In [155]:
df_udemy = pd.read_csv("udemy_ML.csv", header=None, sep=";")
df_udemy["courses"] = [row.split("{")[2] for row in df_udemy[0]]
df_udemy= df_udemy.drop(columns=0,axis=1)
df_udemy["courses"]=df_udemy["courses"].str.replace("'Outcomes':","")
df_udemy["courses"]=df_udemy["courses"].str.replace("[","")
df_udemy["courses"]=df_udemy["courses"].str.replace("]","")
df_udemy["courses"]=df_udemy["courses"].str.replace("}","")
df_udemy["courses"]=df_udemy["courses"].str.replace("\"","'")
df_udemy["courses"]=df_udemy["courses"].apply(lambda x : x + ',')

#df_udemy['courses'] = df_udemy['courses'].apply(lambda x: x.split("'Outcomes': ")[1])

df_udemy.head()

,courses
0,"'Master Machine Learning on Python & R', 'Hav..."
1,'Use Python for Data Science and Machine Lear...
2,'You will learn how to use data science and m...
3,'The course provides the entire toolbox you n...
4,'Learn multiple techniques for missing data i...


In [156]:
first_df = pd.DataFrame(columns=['courses', 'course'])
pattern = "'(.*?)',"
for i, row in enumerate(df_udemy["courses"]):
    lst = re.findall(pattern, row)
    temp_df = pd.DataFrame({'courses': lst, 'course': [f'course {i}']*len(lst)})
    first_df = first_df.append(temp_df, ignore_index=True)

first_df= first_df.drop(columns="course",axis=1)
df = first_df.copy(deep=True)

## Preprocess 

In [157]:
df['courses'] = df['courses'].str.lower()


In [158]:
stop_words = stopwords.words('english')
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    return tokens
# Remove stop words
def remove_stop_words(tokens):
    clean_tokens = [token for token in tokens if token.lower() not in stopwords.words('english')]
    return clean_tokens
def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words
def lemmatize_words(words):
  lemmatizer = WordNetLemmatizer()
  lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
  return lemmatized_words

"""def stem_words(words):
  stemmer = PorterStemmer()
  stemmed_words = [stemmer.stem(word) for word in words]
  return stemmed_words"""

'def stem_words(words):\n  stemmer = PorterStemmer()\n  stemmed_words = [stemmer.stem(word) for word in words]\n  return stemmed_words'

In [159]:
df['courses'] = df['courses'].apply(tokenize)

In [160]:
df.head()

,courses
0,"[master, machine, learning, on, python, &, r]"
1,"[have, a, great, intuition, of, many, machine,..."
2,"[make, accurate, predictions]"
3,"[make, powerful, analysis]"
4,"[make, robust, machine, learning, models]"


In [161]:
df['courses'] = df['courses'].apply(remove_stop_words)
df['courses'] = df['courses'].apply(remove_punctuation)
df['courses'] = df['courses'].apply(lemmatize_words)


In [162]:
df.head()

,courses
0,"[master, machine, learning, python, r]"
1,"[great, intuition, many, machine, learning, mo..."
2,"[make, accurate, prediction]"
3,"[make, powerful, analysis]"
4,"[make, robust, machine, learning, model]"


## Model

In [163]:
corpus = df['courses'] .tolist()

In [ ]:
corpus

### Finding optimal hyperparameters

size: The dimensionality of the word vectors that the model will learn

 window: The size of the context window used to predict the target word

 min_count: The minimum number of times a word must appear in the corpus to be included in the model

 workers: The number of worker threads to use for training the mode

In [165]:
def train_word2vec(corpus, word1, word2, min_size=50, max_size=100, min_window=5, max_window=10, min_count=1, workers=4):
  best_size = min_size
  best_window = min_window
  best_min_count = min_count
  best_model = None
  best_score = -float('inf')

  for size in range(min_size, max_size + 1):
    for window in range(min_window, max_window + 1):
      for min_count in range(1, 3):
        model = Word2Vec(corpus, size=size, window=window, min_count=min_count, workers=workers)
        model.train(corpus, total_examples=len(corpus), epochs=15)
        score = model.wv.similarity(word1, word2)
        if score > best_score:
          best_size = size
          best_window = window
          best_min_count = min_count
          best_model = model
          best_score = score

  print(f'Best size: {best_size}')
  print(f'Best window: {best_window}')
  print(f'Best min_count: {best_min_count}')
  return best_model


Printing top 5 frequent pair of words in corpus for using them finding the optimal hyperparameters.

In [166]:
def most_frequent_pair(corpus):
  # Flatten the corpus list of lists into a single list
  flat_corpus = [word for sublist in corpus for word in sublist]
  # Initialize a counter to count the frequency of each pair of words
  pair_counts = Counter()
  # Iterate through the flat corpus and count the frequency of each pair of words
  for i in range(len(flat_corpus)-1):
    pair = (flat_corpus[i], flat_corpus[i+1])
    pair_counts[pair] += 1
  # Find the top 5 most frequent pairs of words
  top_pairs = pair_counts.most_common(5)
  return top_pairs

print(most_frequent_pair(corpus))

[(('machine', 'learning'), 37), (('neural', 'network'), 16), (('data', 'science'), 11), (('learning', 'algorithm'), 9), (('deep', 'learning'), 8)]


We will use only three of them.
[(('machine', 'learning'), 37), (('neural', 'network'), 16), (('data', 'science'), 11), (('learning', 'algorithm'), 9), (('deep', 'learning'), 8)]


In [167]:
train_word2vec(corpus,'neural','network' ,min_size=50, max_size=100, min_window=1, max_window=10, min_count=1, workers=4)

Best size: 99
Best window: 10
Best min_count: 1


In [168]:
train_word2vec(corpus,'data','science' ,min_size=50, max_size=100, min_window=1, max_window=10, min_count=1, workers=4)

Best size: 55
Best window: 7
Best min_count: 2


In [169]:
train_word2vec(corpus,'machine','learning' ,min_size=50, max_size=100, min_window=1, max_window=10, min_count=1, workers=4)

Best size: 50
Best window: 8
Best min_count: 2


**Results for "neural network" min distance**

Best  size: 99

Best window: 10

Best min_count: 1

**Results for "data science" min distance**

Best size: 55

Best window: 7

Best min_count: 2

**Results for "machine learning" min distance**

Best size: 50

Best window: 8

Best min_count: 2



After compared 3 different similarity we decided to use neural network hyperparameters.

**With neural network hyperparameters**

- Sim(data,scince) 0.82586724

- Sim(neural,network) 0.8188135

- Sim(machine,learning) 0.9418734


**With data science hyperparameters**

- Sim(data,scince) 0.8763663

- Sim(neural,network) 0.7670939

- Sim(machine,learning) 0.946057


**With machine learning hyperparameters**

- Sim(data,scince) 0.7845713

- Sim(neural,network) 0.7333079

- Sim(machine,learning) 0.95651054




In [170]:
model = gensim.models.Word2Vec(corpus, size=99, window=10, min_count=1, workers=4)

In [171]:
model.train(corpus, total_examples=len(corpus), epochs=15)

(15029, 20445)

In [172]:
model.save("course.model")

In [173]:
similarity = model.similarity('data', 'science')
similarity

0.82586724

In [174]:
similarity = model.similarity('neural', 'network')
similarity

0.8188135

In [175]:
similarity = model.similarity('machine', 'learning')
similarity

0.9418734

In [176]:
similar_words = model.wv.most_similar('machine')
similar_words

[('learning', 0.94187331199646),
 ('data', 0.918445348739624),
 ('using', 0.9167494177818298),
 ('service', 0.9072754979133606),
 ('sagemaker', 0.9031240940093994),
 ('algorithm', 0.8981636762619019),
 ('learn', 0.8977909684181213),
 ('model', 0.8946993947029114),
 ('lambda', 0.8853323459625244),
 ('deep', 0.8831156492233276)]

In [177]:
print(len(model.wv["machine"]))
print(model.wv["machine"].shape)
print(model.wv["machine"].ndim)

model.wv["machine"]

99
(99,)
1


array([ 0.00087693,  0.01038807, -0.00729476,  0.01143548,  0.00946426,
       -0.0148143 ,  0.00861648, -0.00800931, -0.00112296,  0.01386101,
       -0.00522516,  0.00872052, -0.0209248 , -0.0022656 ,  0.01280939,
        0.0058496 ,  0.00252772,  0.00720616, -0.00780524,  0.00820428,
       -0.006725  , -0.01446329, -0.00474504, -0.00932457,  0.01245424,
       -0.00581274, -0.00081248,  0.0031515 , -0.01515251, -0.01278656,
        0.01184327, -0.01534771, -0.01302057,  0.00811255,  0.01366294,
        0.00563625,  0.01124899,  0.00454461, -0.01012414, -0.01528313,
       -0.0168926 , -0.00941714, -0.00339623, -0.00543142, -0.00231422,
       -0.00848204,  0.00060684,  0.00439641, -0.00050117, -0.0046956 ,
        0.01627081, -0.0114842 , -0.00508579,  0.009707  , -0.01816428,
       -0.00382941,  0.00679679,  0.00177348, -0.00257985, -0.01031423,
       -0.00790955,  0.00104763, -0.00931221,  0.00161844,  0.00481712,
        0.0003987 ,  0.00145762,  0.00414571, -0.00909115, -0.01

In [178]:
print(len(model.wv["learning"]))
print(model.wv["learning"].shape)
print(model.wv["learning"].ndim)

model.wv["learning"]

99
(99,)
1


array([ 0.00088445,  0.00619289, -0.00587593,  0.01238392,  0.01285887,
       -0.01757634,  0.0095242 , -0.00725863, -0.00465944,  0.0179005 ,
       -0.00318371,  0.00706999, -0.02955654,  0.00369815,  0.01127169,
       -0.00099408,  0.00808878,  0.00792944, -0.00422639,  0.00692396,
       -0.0089468 , -0.02228612, -0.00310032, -0.01012089,  0.01334959,
       -0.00866172,  0.00575668,  0.00634294, -0.01585406, -0.01596242,
        0.01053095, -0.01424923, -0.01002603,  0.00896566,  0.01183014,
        0.00679325,  0.02112303,  0.00159215, -0.01596532, -0.01633585,
       -0.02143682, -0.01050804, -0.00351112, -0.00431512,  0.00258846,
       -0.00717567,  0.00394994,  0.00707728,  0.00465056,  0.00214916,
        0.03043412, -0.01982403, -0.00477893,  0.00453491, -0.02546948,
        0.00486474,  0.01834653,  0.00293592, -0.00324697, -0.00679919,
       -0.00935537,  0.00070363, -0.0127838 , -0.00743295,  0.00392052,
        0.00097816, -0.00098027,  0.00119761, -0.01016978, -0.01